In [1]:
import pandas as pd
df = pd.read_parquet("/kaggle/input/jane-street-real-time-market-data-forecasting/train.parquet/partition_id=0/part-0.parquet")

In [2]:
df.head()

,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,...,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8
0,0,0,1,3.889038,NaN,NaN,NaN,NaN,NaN,0.851033,...,-0.281498,0.738489,-0.069556,1.380875,2.005353,0.186018,1.218368,0.775981,0.346999,0.095504
1,0,0,7,1.370613,NaN,NaN,NaN,NaN,NaN,0.676961,...,-0.302441,2.965889,1.190077,-0.523998,3.849921,2.626981,5.000000,0.703665,0.216683,0.778639
2,0,0,9,2.285698,NaN,NaN,NaN,NaN,NaN,1.056285,...,-0.096792,-0.864488,-0.280303,-0.326697,0.375781,1.271291,0.099793,2.109352,0.670881,0.772828
3,0,0,10,0.690606,NaN,NaN,NaN,NaN,NaN,1.139366,...,-0.296244,0.408499,0.223992,2.294888,1.097444,1.225872,1.225376,1.114137,0.775199,-1.379516
4,0,0,14,0.440570,NaN,NaN,NaN,NaN,NaN,0.955200,...,3.418133,-0.373387,-0.502764,-0.348021,-3.928148,-1.591366,-5.000000,-3.572820,-1.089123,-5.000000


In [3]:
df.columns

Index(['date_id', 'time_id', 'symbol_id', 'weight', 'feature_00', 'feature_01',
       'feature_02', 'feature_03', 'feature_04', 'feature_05', 'feature_06',
       'feature_07', 'feature_08', 'feature_09', 'feature_10', 'feature_11',
       'feature_12', 'feature_13', 'feature_14', 'feature_15', 'feature_16',
       'feature_17', 'feature_18', 'feature_19', 'feature_20', 'feature_21',
       'feature_22', 'feature_23', 'feature_24', 'feature_25', 'feature_26',
       'feature_27', 'feature_28', 'feature_29', 'feature_30', 'feature_31',
       'feature_32', 'feature_33', 'feature_34', 'feature_35', 'feature_36',
       'feature_37', 'feature_38', 'feature_39', 'feature_40', 'feature_41',
       'feature_42', 'feature_43', 'feature_44', 'feature_45', 'feature_46',
       'feature_47', 'feature_48', 'feature_49', 'feature_50', 'feature_51',
       'feature_52', 'feature_53', 'feature_54', 'feature_55', 'feature_56',
       'feature_57', 'feature_58', 'feature_59', 'feature_60', 'feature_6

- `date_id` and `time_id` - Integer values that are ordinally sorted, providing a chronological structure to the data, although the **actual time intervals** between `time_id` values may vary.
- `symbol_id` - Identifies a unique financial instrument. Each row in the dataset corresponds to a unique combination of a symbol (identified by `symbol_id`) and a timestamp (represented by `date_id` and `time_id`). The `symbol_id` column contains encrypted identifiers. Each `symbol_id` is not guaranteed to appear in all `time_id` and `date_id` combinations.
- `weight` - The weighting used for calculating the scoring function.
- `feature_{00...78}` - Anonymized market data.
- `responder_{0...8}` - Anonymized responders clipped between -5 and 5. The responder_6 field is what you are trying to predict.

In [4]:
TEMPORAL_FEATURES = ['date_id', 'time_id','symbol_id']
MARKET_FEATURES = [f'feature_{i:02}' for i in range(0,79)]
RESPONDER_FEATURES = [f'responder_{i}' for i in range(0,9)]
RESPONDER_FEATURES.remove('responder_6')
SYMBOL_FEATURES = ['symbol_id']
WEIGHT = ['WEIGHT']

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1944210 entries, 0 to 1944209
Data columns (total 92 columns):
 #   Column       Dtype  
---  ------       -----  
 0   date_id      int16  
 1   time_id      int16  
 2   symbol_id    int8   
 3   weight       float32
 4   feature_00   float32
 5   feature_01   float32
 6   feature_02   float32
 7   feature_03   float32
 8   feature_04   float32
 9   feature_05   float32
 10  feature_06   float32
 11  feature_07   float32
 12  feature_08   float32
 13  feature_09   int8   
 14  feature_10   int8   
 15  feature_11   int16  
 16  feature_12   float32
 17  feature_13   float32
 18  feature_14   float32
 19  feature_15   float32
 20  feature_16   float32
 21  feature_17   float32
 22  feature_18   float32
 23  feature_19   float32
 24  feature_20   float32
 25  feature_21   float32
 26  feature_22   float32
 27  feature_23   float32
 28  feature_24   float32
 29  feature_25   float32
 30  feature_26   float32
 31  feature_27   float32
 32

In [6]:
df.describe()

,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,...,feature_78,responder_0,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8
count,1.944210e+06,1.944210e+06,1.944210e+06,1.944210e+06,0.0,0.0,0.0,0.0,0.0,1.944210e+06,...,1.944210e+06,1.944210e+06,1.944210e+06,1.944210e+06,1.944210e+06,1.944210e+06,1.944210e+06,1.944210e+06,1.944210e+06,1.944210e+06
mean,9.384629e+01,4.240000e+02,1.376638e+01,1.973281e+00,NaN,NaN,NaN,NaN,NaN,-4.463175e-02,...,-9.805073e-02,8.424639e-03,1.076465e-02,2.412764e-03,1.139089e-02,2.185480e-02,3.326982e-03,1.487634e-03,-4.817980e-04,1.078154e-03
std,4.813196e+01,2.450851e+02,1.108778e+01,9.691969e-01,NaN,NaN,NaN,NaN,NaN,9.479579e-01,...,6.398993e-01,9.558780e-01,1.141793e+00,8.442027e-01,1.276044e+00,1.266448e+00,1.225165e+00,8.705768e-01,8.918150e-01,8.737320e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,4.405696e-01,NaN,NaN,NaN,NaN,NaN,-1.176608e+01,...,-3.393299e+00,-5.000000e+00,-5.000000e+00,-5.000000e+00,-5.000000e+00,-5.000000e+00,-5.000000e+00,-5.000000e+00,-5.000000e+00,-5.000000e+00
25%,5.400000e+01,2.120000e+02,7.000000e+00,1.323803e+00,NaN,NaN,NaN,NaN,NaN,-4.756072e-01,...,-3.195696e-01,-2.322211e-01,-2.667868e-01,-1.210319e-01,-4.444537e-01,-5.146699e-01,-2.569897e-01,-3.558709e-01,-3.914038e-01,-2.892584e-01
50%,9.900000e+01,4.240000e+02,1.200000e+01,1.763827e+00,NaN,NaN,NaN,NaN,NaN,-5.818180e-02,...,-2.468792e-01,-3.943805e-03,-2.333469e-02,-1.248489e-03,-1.031224e-02,-1.836913e-02,-4.690550e-03,-9.597129e-03,-2.376243e-02,-3.107830e-04
75%,1.360000e+02,6.360000e+02,1.700000e+01,2.393846e+00,NaN,NaN,NaN,NaN,NaN,3.493771e-01,...,-1.256560e-01,2.309678e-01,2.538152e-01,1.191232e-01,4.292141e-01,5.065366e-01,2.438874e-01,3.360999e-01,3.463979e-01,2.840819e-01
max,1.690000e+02,8.480000e+02,3.800000e+01,6.011999e+00,NaN,NaN,NaN,NaN,NaN,1.532000e+01,...,4.370195e+01,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00,5.000000e+00


In [7]:
empty_columns = []
fully_filled_columns = []
partially_empty_columns = []

In [8]:
for feature in list(df.columns):
    empty_rows = sum(df[[feature]].isna().values)[0]
    nonempty_rows = len(df[[feature]]) - empty_rows

    ## store in feature_stats since recalculating empty rows for all 79 features takes a while
    if nonempty_rows == 0:
        empty_columns.append(feature)

    elif empty_rows == 0:
        fully_filled_columns.append(feature)

    else:
        partially_empty_columns.append(feature)
    
    print(f'{feature} : total  - {len(df[[feature]])} - empty - {empty_rows} - nonempty - {nonempty_rows}')

date_id : total  - 1944210 - empty - 0 - nonempty - 1944210
time_id : total  - 1944210 - empty - 0 - nonempty - 1944210
symbol_id : total  - 1944210 - empty - 0 - nonempty - 1944210
weight : total  - 1944210 - empty - 0 - nonempty - 1944210
feature_00 : total  - 1944210 - empty - 1944210 - nonempty - 0
feature_01 : total  - 1944210 - empty - 1944210 - nonempty - 0
feature_02 : total  - 1944210 - empty - 1944210 - nonempty - 0
feature_03 : total  - 1944210 - empty - 1944210 - nonempty - 0
feature_04 : total  - 1944210 - empty - 1944210 - nonempty - 0
feature_05 : total  - 1944210 - empty - 0 - nonempty - 1944210
feature_06 : total  - 1944210 - empty - 0 - nonempty - 1944210
feature_07 : total  - 1944210 - empty - 0 - nonempty - 1944210
feature_08 : total  - 1944210 - empty - 16980 - nonempty - 1927230
feature_09 : total  - 1944210 - empty - 0 - nonempty - 1944210
feature_10 : total  - 1944210 - empty - 0 - nonempty - 1944210
feature_11 : total  - 1944210 - empty - 0 - nonempty - 1944210

So we have 3 cases
1. Fully Empty columns : we can just drop them
2. Partially Empty Columns: we can impute them
3. Full Columns : No imputation necessary

In [ ]:
df.sort_values(['time_id','date_id'])

In [ ]:
df[df.date_id==0].symbol_id.unique()

In [ ]:
df[df.date_id==1].symbol_id.unique()

In [ ]:
df[df.symbol_id == 1].date_id.unique()

In [ ]:
set(df[df.symbol_id == 1].date_id.unique()) - set(df[df.symbol_id == 7].date_id.unique()) 

In [ ]:
mask1 = (df.symbol_id == 1) & (df.date_id == 0)
mask2 = (df.symbol_id == 7) & (df.date_id == 0)
set(df[mask1].time_id.unique()) - set(df[mask2].date_id.unique()) 

## Splitting Temporally

In [33]:
df = df.sort_values(['date_id', 'time_id'])
date_counts = df.date_id.value_counts()

In [34]:
date_counts = pd.DataFrame(date_counts.sort_index())
date_counts['cumulative_sum'] = date_counts['count'].cumsum()
date_counts.head()

,count,cumulative_sum
date_id,,
0,6792,6792
1,10188,16980
2,9339,26319
3,10188,36507
4,10188,46695


We want to do a 80-20 split based on date

In [35]:
total = len(df)
split = 0.8
apprx_train_len = int(total*split)
apprx_test_len = total - apprx_train_len
date_counts['Train'] = date_counts['cumulative_sum'] < apprx_train_len
date_counts.head()

,count,cumulative_sum,Train
date_id,,,
0,6792,6792,True
1,10188,16980,True
2,9339,26319,True
3,10188,36507,True
4,10188,46695,True


In [41]:
last_train_data = date_counts[date_counts.Train == True].tail(1) 
last_train_data

,count,cumulative_sum,Train
date_id,,,
142,16131,1546029,True


In [42]:
first_test_data = date_counts[date_counts.Train == False].head(1)
first_test_data

,count,cumulative_sum,Train
date_id,,,
143,15282,1561311,False


In [51]:
train_len = last_train_data['cumulative_sum'].values[0]
test_len = len(df) - last_train_data['cumulative_sum'].values[0]
print("Training data :", train_len,"-" ,round(train_len*100/len(df),2), "%" )
print("Testing data :", test_len, "-", round(test_len*100/len(df),2), "%")

Training data : 1546029 - 79.52 %
Testing data : 398181 - 20.48 %


In [60]:
df.loc[:, 'Train'] = df.date_id <= last_train_data.index[0]
df.head()

,date_id,time_id,symbol_id,weight,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,...,responder_1,responder_2,responder_3,responder_4,responder_5,responder_6,responder_7,responder_8,Split,Train
0,0,0,1,3.889038,NaN,NaN,NaN,NaN,NaN,0.851033,...,-0.069556,1.380875,2.005353,0.186018,1.218368,0.775981,0.346999,0.095504,True,True
1,0,0,7,1.370613,NaN,NaN,NaN,NaN,NaN,0.676961,...,1.190077,-0.523998,3.849921,2.626981,5.000000,0.703665,0.216683,0.778639,True,True
2,0,0,9,2.285698,NaN,NaN,NaN,NaN,NaN,1.056285,...,-0.280303,-0.326697,0.375781,1.271291,0.099793,2.109352,0.670881,0.772828,True,True
3,0,0,10,0.690606,NaN,NaN,NaN,NaN,NaN,1.139366,...,0.223992,2.294888,1.097444,1.225872,1.225376,1.114137,0.775199,-1.379516,True,True
4,0,0,14,0.440570,NaN,NaN,NaN,NaN,NaN,0.955200,...,-0.502764,-0.348021,-3.928148,-1.591366,-5.000000,-3.572820,-1.089123,-5.000000,True,True


In [ ]:
df = df.drop_columns(empty_columns)
train_df = df[df.Train == True]
test_df = df[df.Train == False]
# train_df.to_csv('train.csv', index = False)
# test_df.to_csv('test.csv', index = False)

In [ ]:
# %load_ext cudf.pandas
# import pandas as pdd